In [ ]:
import numpy as np
from daskms import xds_from_table,xds_from_ms
import svdcompressor
import svdaverager
from casacore.tables import table
import warnings
import msodify
import dask.array as da
import compress


In [2]:
msfile = '/home/mukundi/averaging/stimela_scripts/msdir/sim-svd-test.ms'
mssub = '/home/mukundi/averaging/stimela_scripts/msdir/sim-simms.ms::ANTENNA'

In [3]:
antlist = [[0,1],
           [1,2],
           ]
compress.compress_visdata(msfile,correlation='XX,XY,YX,YY',compressionrank=1,outcol='SV1_DATA',antlist=antlist,zarr_output_path='/home/mukundi/bdsvd/')

11:04 14:44:28-BDSVD-INFO    | Decomposing visibilities for DATA_DESC_ID 0. FIELD_ID 0, and SCAN_NUMBER 1.
11:04 14:44:28-BDSVD-INFO    | The following correlations are available:['XX', 'XY', 'YX', 'YY']
11:04 14:44:28-BDSVD-INFO    | The user has selected the following correlations:['XX', 'XY', 'YX', 'YY']
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation XX
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation XY
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation YX
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation YY
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M001-M002 and correlation XX
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M001-M002 and correlation XY
11:04 14:44:28-BDSVD-INFO    | Decomposing visibility data for baseline M001-M002 and

11:04 14:44:32-BDSVD-ERROR   | Failed to write to Zarr format at /home/mukundi/bdsvd/: Could not find a Chunk Manager which recognises type <class 'dask.array.core.Array'>
11:04 14:44:32-BDSVD-INFO    | Visibility data has been compressed in column DATA from data size of 6.79e-05 GB to a compressed data size of -2.10e-05 GB


(/code/casa/OS/Path.cc : 407) Failed AlwaysAssert getcwd(temp, 1024)
(/code/casa/OS/Path.cc : 407) Failed AlwaysAssert getcwd(temp, 1024)


In [10]:
maintable = xds_from_table(msfile)[0]#,taql_where=(f"DATA_DESC_ID=0 AND SCAN_NUMBER=1 AND FIELD_ID=0 AND ANTENNA1=0 AND ANTENNA2=1"))[0]
# tabl = table(msfile,readonly=False,lockoptions='auto')
subtable = xds_from_table(mssub)[0]
ant1_mask = (maintable.ANTENNA1.data.compute() == 0)
ant2_mask = (maintable.ANTENNA2.data.compute() == 5)
fieldidmask = (maintable.FIELD_ID.data.compute()==0)
filter = ant1_mask & ant2_mask
baseline_data = maintable.COMPRESSED_DATA.data[:,:,3]
data = maintable.DATA.data[:,:,3]


In [11]:
baseline_data.compute()[0,20], data.compute()[0,20]

((1.0002928972244263+0.00872578751295805j), (1.0002929+0.0087257875j))

In [14]:
subtable.NAME.values[0]

'M000'

In [ ]:
np.unique(list(zip([5,6],[5,7])))
a = zip([5,6],[5,7])
a = list(a)
da.unique(np.array(a)).compute()

In [ ]:
tab.DATA_DESC_ID.data.compute()

In [ ]:
ranks = [4,5,6,7]

for i in ranks:
    run = svdaverager.uniform_svd(msfile,rank=i,column='DATA',correlation='XX,YY', outcol=f"{i}SV_DATA")

In [30]:
# comp = tab.COMPRESSED_DATA.data.compute()[:,:,0]
data = tab.CORRECTED_DATA.data.compute()[:,:,0]
bs = tab.DATA.data.compute()[::2015,:,0]
cf3 = tab.MODEL_DATA.data.compute()[:,:,0]
dec999 = tab.BDSVD_DATA.data.compute()[::2016,:,0]
# td = tab.TEST_DATA.data.compute()[:,:,0]
rr = run.compressed_data.compute()[::2015,:,0]

In [ ]:
tab

In [31]:
signal = np.linalg.norm(rr-bs, 'fro') / np.linalg.norm(bs, 'fro')